# ВАРИАНТ №18
* Наименование показателя X: Число больничных коек на 10 тысяч населения
* Фелеральный округ (ФО): Центральный ФО
* a0: Среднее значение показателя по ФО
* alpha: 0,06
* Критерии проверки на нормальность: Шапиро-Уилка, Колмогорова-Смирнова, Хи-квадрат

In [23]:
import numpy as np
import pandas as pd
from scipy.stats import kstest
from scipy.stats import bartlett
from scipy.stats import shapiro
from scipy.stats import ttest_ind
from scipy.stats import ttest_1samp
from scipy.stats import f_oneway
from scipy.stats import chisquare
from scipy.stats import norm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statistics as st
import matplotlib.pyplot as plt

1

In [45]:
pd.options.display.max_rows = 20
doc2000 = pd.read_excel("/content/СтатАнализТабл.xlsx",index_col='год')
doc2000

,Центральный федеральный округ,Белгородская область,Брянская область,Владимирская область,Воронежская область,Ивановская область,Калужская область,Костромская область,Курская область,Липецкая область,...,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40
год,,,,,,,,,,,,,,,,,,,,,
1990,139.6,147.2,140.9,144.3,134.6,149.7,133.8,157.5,146.7,144.8,...,99.9,75.2,102.1,99.2,110.3,100.3,108.4,106.2,115.2,93.4
1991,137.3,145.7,139.9,137.0,131.7,146.1,129.9,156.6,141.6,129.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992,134.2,143.4,132.7,130.9,129.5,146.1,126.8,155.5,133.4,140.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993,131.8,141.3,127.4,130.1,133.3,146.7,123.9,154.1,133.9,137.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994,130.3,135.2,126.8,130.5,127.9,145.7,132.3,154.6,124.8,134.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995,128.1,131.6,126.0,129.5,127.5,146.9,128.6,155.0,121.6,134.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,127.6,128.3,124.9,128.8,125.9,149.3,123.8,154.9,119.7,134.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,125.6,126.4,123.5,128.3,125.3,148.6,120.6,154.8,120.2,130.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,123.7,126.4,122.5,125.6,119.8,144.4,115.3,153.2,119.5,130.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


2

In [46]:
X = [0] * 19
X[0] = doc2000['Центральный федеральный округ']
X[1] = doc2000['Белгородская область']
X[2] = doc2000['Брянская область']
X[3] = doc2000['Владимирская область']
X[4] = doc2000['Воронежская область']
X[5] = doc2000['Ивановская область']
X[6] = doc2000['Калужская область']
X[7] = doc2000['Костромская область']
X[8] = doc2000['Курская область']
X[9] = doc2000['Липецкая область']
X[10] = doc2000['Московская область']
X[11] = doc2000['Орловская область']
X[12] = doc2000['Рязанская область']
X[13] = doc2000['Смоленская область']
X[14] = doc2000['Тамбовская область']
X[15] = doc2000['Тверская область']
X[16] = doc2000['Тульская область']
X[17] = doc2000['Ярославльская область']
X[18] = doc2000['г. Москва']


X[0].describe()
#plt.plot(X[1])
#X[0] = pd.Series(X[0])
#X[0].hist(density=True)
#shapiro(X[0])
#kstest(X[2],"norm", args=[120, 10])

count     20.000000
mean     121.590000
std       10.929482
min       98.800000
25%      114.600000
50%      122.000000
75%      128.650000
max      139.600000
Name: Центральный федеральный округ, dtype: float64

In [47]:
alpha = 0.06
Y = [0] * 19
j = 0

def found_freqs(t, a, sigma):
  f_obs =[]
  f_exp=[]
  tmp=np.array(t)
  tmp_bins=pd.cut(tmp, 5)
  df = pd.DataFrame(tmp_bins.describe())
  f_obs=np.array(df['counts'])
  categories = tmp_bins.categories
  #print(df)
  n=len(categories)
  inter=[0.0]*(n+1)

  for i in range(n):
    inter[i]=categories[i].left
  inter[n]=categories[n-1].right
  #print(inter)

  f_exp =[0.0]*n
  for i in range(n):
    f_exp[i]=norm.cdf((inter[i+1]-a)/sigma) - norm.cdf((inter[i]-a)/sigma)

  f_exp=[i * 20 for i in f_exp] #умножаем на размер выборки

  f = 20.0 - sum(f_exp)
  f = f/5

  f_exp=[i+f for i in f_exp]
  f_exp=np.array(f_exp)
  return f_obs, f_exp



for i in range(18):
  
  print(X[i].name)
  tmp1 = shapiro(X[i])[1]
  if(tmp1 > alpha):
    print("Shapiro: normal")
  else:
    print("Shapiro: not normal")

  tmp2 = kstest(X[i],"norm", args=[X[i].mean(),X[i].std()])[1]
  if(tmp2 > alpha):
    print("Ktest: normal")
  else:
    print("Ktest: not normal")
  
  a = X[i].mean()
  sigma=np.sqrt(st.variance(X[i]))
  f_obs, f_exp = found_freqs(X[i], a, sigma)
  tmp3 = chisquare(f_obs, f_exp)[1]
  if(tmp3 > alpha):
    print("Chisquare: normal")
  else:
    print("Chisquare: not normal")
  print()

  if tmp1 > alpha or tmp2 > alpha or tmp3 > alpha:
    Y[j]=X[i]
    j += 1
Y = np.resize(Y, j)


Центральный федеральный округ
Shapiro: normal
Ktest: normal
Chisquare: normal

Белгородская область
Shapiro: not normal
Ktest: normal
Chisquare: normal

Брянская область
Shapiro: not normal
Ktest: not normal
Chisquare: not normal

Владимирская область
Shapiro: normal
Ktest: normal
Chisquare: normal

Воронежская область
Shapiro: normal
Ktest: normal
Chisquare: normal

Ивановская область
Shapiro: not normal
Ktest: normal
Chisquare: not normal

Калужская область
Shapiro: normal
Ktest: normal
Chisquare: normal

Костромская область
Shapiro: not normal
Ktest: normal
Chisquare: not normal

Курская область
Shapiro: normal
Ktest: normal
Chisquare: normal

Липецкая область
Shapiro: not normal
Ktest: not normal
Chisquare: not normal

Московская область
Shapiro: not normal
Ktest: normal
Chisquare: normal

Орловская область
Shapiro: normal
Ktest: normal
Chisquare: normal

Рязанская область
Shapiro: not normal
Ktest: normal
Chisquare: normal

Смоленская область
Shapiro: not normal
Ktest: normal
Chis

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:1822: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asanyarray(a).ravel(order=order)


In [48]:
for i in range(len(Y)):
  print(Y[i].name)

Центральный федеральный округ
Белгородская область
Владимирская область
Воронежская область
Ивановская область
Калужская область
Костромская область
Курская область
Московская область
Орловская область
Рязанская область
Смоленская область
Тамбовская область
Тверская область
Ярославльская область


3

In [57]:
from scipy import stats
_list = []
for i in range(1, len(X)):
  count = 0
  for j in range(i+1, 17, 1):
    if(stats.bartlett(X[i], X[j])[1] > alpha):
      count = count + 1
  _list.append(count)

max = np.max(_list)
ind = 0
for i in range(1, len(X)-1):
  if(_list[i]==max):
    ind = i
_list = []
_list_ind = []
_list.append(X[ind])
_list_ind.append(ind)

for j in range(ind+1, len(X)-1):
  if(stats.bartlett(X[ind], X[j])[1] > alpha):
    _list.append(X[j])
    _list_ind.append(j)

print(_list_ind)
for k in _list_ind:
  print(X[k].name)

[1, 3, 5, 8, 10]
Белгородская область
Владимирская область
Ивановская область
Курская область
Московская область


4

In [69]:
region = _list[0]
neighbour = _list[3]
compare_disper = bartlett(region, neighbour)
compare_mean = ttest_ind(region, neighbour)
if compare_mean[1] > alpha and compare_disper[1] > alpha:
  print(region.name, "и", neighbour.name)
  print(bartlett(region, neighbour))

Белгородская область и Курская область
BartlettResult(statistic=2.5205690980984246, pvalue=0.11237002450289219)


5

In [70]:
a0 = np.mean(X[0])
test = ttest_1samp(region, popmean= a0, alternative='less')
print(test)
if (test[1] > alpha):
  print("В среднем статистически ниже, чем a0")
else:
  print("В среднем статистически выше, чем a0")

Ttest_1sampResult(statistic=-0.027702896087245243, pvalue=0.48909401245094986)
В среднем статистически ниже, чем a0


6

In [71]:
test = ttest_1samp(neighbour, popmean= a0, alternative='two-sided')
print(test)
if (test[1] > alpha):
  print("В среднем статистически отличается от a0")
else:
  print("В среднем статистически не отличается от a0")

Ttest_1sampResult(statistic=-1.0460982980389768, pvalue=0.3086423401302545)
В среднем статистически отличается от a0


7

In [72]:
reg = X[1] + X[3] + X[5] + X[8] + X[10]

dat1 = {'val': reg, 'info': np.repeat([X[1].name, X[3].name, X[5].name, X[8].name, X[10].name], repeats= 4)}

dat = pd.DataFrame(data=dat1)

tueky_out = pairwise_tukeyhsd(endog=dat['val'] , groups=dat['info'], alpha=0.06)
print(tueky_out)

                Multiple Comparison of Means - Tukey HSD, FWER=0.06                 
       group1               group2        meandiff p-adj    lower     upper   reject
------------------------------------------------------------------------------------
Белгородская область Владимирская область   -58.85 0.0333 -112.0998   -5.6002   True
Белгородская область   Ивановская область  -92.425  0.001 -145.6748  -39.1752   True
Белгородская область      Курская область  -119.65  0.001 -172.8998  -66.4002   True
Белгородская область   Московская область   -215.6  0.001 -268.8498 -162.3502   True
Владимирская область   Ивановская область  -33.575 0.3664  -86.8248   19.6748  False
Владимирская область      Курская область    -60.8  0.027 -114.0498   -7.5502   True
Владимирская область   Московская область  -156.75  0.001 -209.9998 -103.5002   True
  Ивановская область      Курская область  -27.225 0.5553  -80.4748   26.0248  False
  Ивановская область   Московская область -123.175  0.001 -176.42